# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
working_directory = 'GitHub/dl-superresolution-ipynb'
%cd /content/drive/MyDrive/$working_directory
!git status

In [ ]:
!pip install -r requirements.txt

## Git Management

In [ ]:
!git config --global user.email "e11909390@student.tuwien.ac.at"
!git config --global user.name "Lollobin"

In [ ]:
!git pull

In [ ]:
!git status

In [ ]:
!git commit -a -m "added connection for google colab"

In [ ]:
!git push

# Preparation

## Data Preprocessing

Generate patches for training and low res images for validation and testing.

### Patch Generation

In [97]:
from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt
import patchify
import numpy as np
import matplotlib.gridspec as gridspec
import glob as glob
import os
import cv2

SHOW_PATCHES = False
STRIDE = 14
SIZE = 32

def show_patches(patches):
    plt.figure(figsize=(patches.shape[0], patches.shape[1]))
    gs = gridspec.GridSpec(patches.shape[0], patches.shape[1])
    gs.update(wspace=0.01, hspace=0.02)
    counter = 0
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            ax = plt.subplot(gs[counter])
            plt.imshow(patches[i, j, 0, :, :, :])
            plt.axis('off')
            counter += 1
    plt.show()

def create_patches(
    input_paths, out_hr_path, out_lr_path,
):
    os.makedirs(out_hr_path, exist_ok=True)
    os.makedirs(out_lr_path, exist_ok=True)
    all_paths = []
    for input_path in input_paths:
        all_paths.extend(glob.glob(f"{input_path}/*"))
    print(f"Creating patches for {len(all_paths)} images")
    for image_path in tqdm(all_paths, total=len(all_paths)):
        image = Image.open(image_path)
        image_name = image_path.split(os.path.sep)[-1].split('.')[0]
        w, h = image.size
        # Create patches of size (32, 32, 3)
        patches = patchify.patchify(np.array(image), (32, 32, 3), STRIDE)
        if SHOW_PATCHES:
            show_patches(patches)
        counter = 0
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):
                counter += 1
                patch = patches[i, j, 0, :, :, :]
                patch = cv2.cvtColor(patch, cv2.COLOR_RGB2BGR)
                cv2.imwrite(
                    f"{out_hr_path}/{image_name}_{counter}.png",
                    patch
                )
                # Convert to bicubic and save.
                h, w, _ = patch.shape
                low_res_img = cv2.resize(patch, (int(w*0.5), int(h*0.5)),
                                        interpolation=cv2.INTER_CUBIC)
                # Now upscale using BICUBIC.
                high_res_upscale = cv2.resize(low_res_img, (w, h),
                                            interpolation=cv2.INTER_CUBIC)
                cv2.imwrite(
                    f"{out_lr_path}/{image_name}_{counter}.png",
                    high_res_upscale
                )

In [ ]:
create_patches(['input/T91'], 'input/t91_hr_patches', 'input/t91_lr_patches')

### Bicubic Scaling for Validation

In [ ]:
from PIL import Image
import glob as glob
import os

paths = ['input/Set14/original', 'input/Set5/original']
scale_factor = '2x' # options 2x, 3x, 4x
images = []

for path in paths:
    images.extend(glob.glob(f"{path}/*.png"))
print(len(images))
# Select scaling-factor and set up directories according to that.
if scale_factor == '2x':
    scale_factor = 0.5
    os.makedirs('input/test_bicubic_rgb_2x', exist_ok=True)
    save_path_lr = 'input/test_bicubic_rgb_2x'
    os.makedirs('input/test_hr', exist_ok=True)
    save_path_hr = 'input/test_hr'
if scale_factor == '3x':
    scale_factor = 0.333
    os.makedirs('input/test_bicubic_rgb_3x', exist_ok=True)
    os.makedirs('input/test_hr', exist_ok=True)
    save_path_lr = 'input/test_bicubic_rgb_3x'
    save_path_hr = 'input/test_hr'
if scale_factor == '4x':
    scale_factor = 0.25
    os.makedirs('input/test_bicubic_rgb_4x', exist_ok=True)
    os.makedirs('input/test_hr', exist_ok=True)
    save_path_lr = 'input/test_bicubic_rgb_4x'
    save_path_hr = 'input/test_hr'
print(f"Scaling factor: {scale_factor}")
print(f"Low resolution images save path: {save_path_lr}")
for image in images:
    orig_img = Image.open(image)
    image_name = image.split(os.path.sep)[-1]
    w, h = orig_img.size[:]
    print(f"Original image dimensions: {w}, {h}")
    orig_img.save(f"{save_path_hr}/{image_name}")
    low_res_img = orig_img.resize((int(w*scale_factor), int(h*scale_factor)), Image.BICUBIC)
    # Upscale using BICUBIC.
    high_res_upscale = low_res_img.resize((w, h), Image.BICUBIC)
    high_res_upscale.save(f"{save_path_lr}/{image_name}")

## Utils

Define utility functions that are used later on.

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision.utils import save_image
plt.style.use('ggplot')
def psnr(label, outputs, max_val=1.):
    """
    Compute Peak Signal to Noise Ratio (the higher the better).
    PSNR = 20 * log10(MAXp) - 10 * log10(MSE).
    https://en.wikipedia.org/wiki/Peak_signal-to-noise_ratio#Definition
    Note that the output and label pixels (when dealing with images) should
    be normalized as the `max_val` here is 1 and not 255.
    """
    label = label.cpu().detach().numpy()
    outputs = outputs.cpu().detach().numpy()
    diff = outputs - label
    rmse = math.sqrt(np.mean((diff) ** 2))
    if rmse == 0:
        return 100
    else:
        PSNR = 20 * math.log10(max_val / rmse)
        return PSNR

def save_plot(train_loss, val_loss, train_psnr, val_psnr):
    # Loss plots.
    plt.figure(figsize=(10, 7))
    plt.plot(train_loss, color='orange', label='train loss')
    plt.plot(val_loss, color='red', label='validataion loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('outputs/loss.png')
    plt.close()
    # PSNR plots.
    plt.figure(figsize=(10, 7))
    plt.plot(train_psnr, color='green', label='train PSNR dB')
    plt.plot(val_psnr, color='blue', label='validataion PSNR dB')
    plt.xlabel('Epochs')
    plt.ylabel('PSNR (dB)')
    plt.legend()
    plt.savefig('outputs/psnr.png')
    plt.close()

def save_model_state(model):
    # save the model to disk
    print('Saving model...')
    torch.save(model.state_dict(), 'outputs/model.pth')

def save_model(epochs, model, optimizer, criterion):
    """
    Function to save the trained model to disk.
    """
    # Remove the last model checkpoint if present.
    torch.save({
                'epoch': epochs+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                }, f"outputs/model_ckpt.pth")

def save_validation_results(outputs, epoch, batch_iter):
    """
    Function to save the validation reconstructed images.
    """
    save_image(
        outputs,
        f"outputs/valid_results/val_sr_{epoch}_{batch_iter}.png"
    )

# Data Loading

In [2]:
import torch
import numpy as np
import glob as glob
from torch.utils.data import DataLoader, Dataset
from PIL import Image

TRAIN_BATCH_SIZE = 128
TEST_BATCH_SIZE = 1

# The SRCNN dataset module.
class SRCNNDataset(Dataset):
    def __init__(self, image_paths, label_paths):
        self.all_image_paths = glob.glob(f"{image_paths}/*")
        self.all_label_paths = glob.glob(f"{label_paths}/*") 
    
    def __len__(self):
        return (len(self.all_image_paths))
    
    def __getitem__(self, index):
        image = Image.open(self.all_image_paths[index]).convert('RGB')
        label = Image.open(self.all_label_paths[index]).convert('RGB')
        image = np.array(image, dtype=np.float32)
        label = np.array(label, dtype=np.float32)
        image /= 255.
        label /= 255.
        image = image.transpose([2, 0, 1])
        label = label.transpose([2, 0, 1])
        return (
            torch.tensor(image, dtype=torch.float),
            torch.tensor(label, dtype=torch.float)
        )
    
# Prepare the datasets.
def get_datasets(
    train_image_paths, train_label_paths,
    valid_image_path, valid_label_paths
):
    dataset_train = SRCNNDataset(
        train_image_paths, train_label_paths
    )
    dataset_valid = SRCNNDataset(
        valid_image_path, valid_label_paths
    )
    return dataset_train, dataset_valid

# Prepare the data loaders
def get_dataloaders(dataset_train, dataset_valid):
    train_loader = DataLoader(
        dataset_train, 
        batch_size=TRAIN_BATCH_SIZE,
        shuffle=True
    )
    valid_loader = DataLoader(
        dataset_valid, 
        batch_size=TEST_BATCH_SIZE,
        shuffle=False
    )
    return train_loader, valid_loader

In [71]:
import torch
import numpy as np
import glob as glob
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from torchvision import transforms

TRAIN_BATCH_SIZE = 128
TEST_BATCH_SIZE = 1

class ResNetSRCNNDataset(Dataset):
    def __init__(self, image_paths, label_paths, transform=None):
        self.all_image_paths = glob.glob(f"{image_paths}/*")
        self.all_label_paths = glob.glob(f"{label_paths}/*")
        self.transform = transform 
    
    def __len__(self):
        return (len(self.all_image_paths))
    
    def __getitem__(self, index):
        image = Image.open(self.all_image_paths[index]).convert('RGB')
        label = Image.open(self.all_label_paths[index]).convert('RGB')

        if self.transform:
            image = self.transform(image)
            label = self.transform(label)

        return image, label
    

class ResizeToEven:
    def __call__(self, image):
        # Get current size
        width, height = image.size
        
        # If the width or height is odd, reduce by one pixel
        new_width = width if width % 2 == 0 else width - 1
        new_height = height if height % 2 == 0 else height - 1
        
        # Resize the image if needed
        if new_width != width or new_height != height:
            image = image.resize((new_width, new_height), Image.BICUBIC)
        
        return image



preprocess_train =  transforms.Compose([
        #transforms.Resize(224, interpolation=transforms.InterpolationMode.BICUBIC),
        ResizeToEven(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

preprocess_valid =  transforms.Compose([
        #transforms.Resize(224, interpolation=transforms.InterpolationMode.BICUBIC),
        ResizeToEven(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
# Prepare the datasets.
def get_datasets(
    train_image_paths, train_label_paths,
    valid_image_path, valid_label_paths
):
    dataset_train = ResNetSRCNNDataset(
        train_image_paths, train_label_paths, transform=preprocess_train
    )
    dataset_valid = ResNetSRCNNDataset(
        valid_image_path, valid_label_paths, transform=preprocess_valid
    )
    return dataset_train, dataset_valid

# Prepare the data loaders
def get_dataloaders(dataset_train, dataset_valid):
    train_loader = DataLoader(
        dataset_train, 
        batch_size=TRAIN_BATCH_SIZE,
        shuffle=True
    )
    valid_loader = DataLoader(
        dataset_valid, 
        batch_size=TEST_BATCH_SIZE,
        shuffle=False
    )
    return train_loader, valid_loader



In [7]:
%matplotlib inline

In [ ]:
from torchvision import utils

def imshow(img):
    img = img.numpy().transpose((1,2,0))

    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1)

    plt.imshow(img)

TRAIN_LABEL_PATHS = 'input/t91_hr_patches'
TRAN_IMAGE_PATHS = 'input/t91_lr_patches'
VALID_LABEL_PATHS = 'input/test_hr'
VALID_IMAGE_PATHS = 'input/test_bicubic_rgb_2x'

dataset_train, dataset_valid = get_datasets(
    TRAN_IMAGE_PATHS, TRAIN_LABEL_PATHS,
    VALID_IMAGE_PATHS, VALID_LABEL_PATHS
)
train_loader, valid_loader = get_dataloaders(dataset_train, dataset_valid)

#images = next(iter(train_loader))
images = next(iter(valid_loader))

#out = utils.make_grid(images)
imshow(images[1][0])

# Network Definition

## SRCNN

In [92]:
import torch.nn as nn
import torch.nn.functional as F

class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(
            3, 64, kernel_size=9, stride=(1, 1), padding=(2, 2)
        )
        self.conv2 = nn.Conv2d(
            64, 32, kernel_size=1, stride=(1, 1), padding=(2, 2)
        )
        self.conv3 = nn.Conv2d(
            32, 3, kernel_size=5, stride=(1, 1), padding=(2, 2)
        )
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)

        return x

## ResNet SRCNN Combination

In [43]:
import torch
import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights

# Load a pretrained ResNet model
resnet = resnet18(weights=ResNet18_Weights.DEFAULT)

class ResNetSRCNN(nn.Module):
    def __init__(self):
        super(ResNetSRCNN, self).__init__()
        
        # Use only the initial layers of ResNet without downsampling
        self.resnet_layers = nn.Sequential(
            resnet.conv1,  # First convolutional layer
            resnet.bn1,    # Batch normalization
            resnet.relu,   # Activation
            resnet.layer1  # First residual block (without downsampling)
        )
        
        # Freeze the ResNet layers
        for param in self.resnet_layers.parameters():
            param.requires_grad = False
        
        # SRCNN-inspired layers for feature extraction and reconstruction
        self.reconstruction = nn.Sequential(
            nn.ConvTranspose2d(64,64,kernel_size=4,stride=2,padding=1), # Upscaling to keep input and output size the same
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),  # Keep the spatial size same
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 3, kernel_size=3, padding=1),   # Output RGB channels
        )

    def forward(self, x):
        # Feature extraction (ResNet initial layers)
        features = self.resnet_layers(x)
        
        # Reconstruction (SRCNN-inspired layers)
        x = self.reconstruction(features)
        
        return x

In [ ]:
# Create the model
resNetSRCNN = ResNetSRCNN()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resNetSRCNN = resNetSRCNN.to(device)

# Example forward pass with a dummy input
dummy_input = torch.randn(1, 3, 18,18).to(device)

resnet_output = resNetSRCNN.resnet_layers(dummy_input)
output = resNetSRCNN(dummy_input)

# Output shape should match the input shape
print(f"Input shape: {dummy_input.shape}")
print(f"ResNet output shape: {resnet_output.shape}")
print(f"Final output shape: {output.shape}")
#print(resNetSRCNN)

# Training

In [ ]:
import torch
import time
import torch.optim as optim
import torch.nn as nn
import os
from tqdm import tqdm

torch.cuda.is_available()

In [ ]:
# Learning parameters.
epochs = 10 # Number of epochs to train the SRCNN model for.
lr = 0.001 # Learning rate.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
weights = None # weights/checkpoint path to resume training

# Constants
TRAIN_LABEL_PATHS = 'input/t91_hr_patches'
TRAN_IMAGE_PATHS = 'input/t91_lr_patches'
VALID_LABEL_PATHS = 'input/test_hr'
VALID_IMAGE_PATHS = 'input/test_bicubic_rgb_2x'
SAVE_VALIDATION_RESULTS = True

os.makedirs('outputs/valid_results', exist_ok=True)

# Initialize the model.
print('Computation device: ', device)
model = ResNetSRCNN().to(device)
if weights is not None:
    print('Loading weights to resume training...')
    checkpoint = torch.load(weights)
    model.load_state_dict(checkpoint['model_state_dict'])
print(model)

# Optimizer.
optimizer = optim.Adam(model.parameters(), lr=lr)

# Loss function. 
criterion = nn.MSELoss()

dataset_train, dataset_valid = get_datasets(
    TRAN_IMAGE_PATHS, TRAIN_LABEL_PATHS,
    VALID_IMAGE_PATHS, VALID_LABEL_PATHS
)
train_loader, valid_loader = get_dataloaders(dataset_train, dataset_valid)

print(f"Training samples: {len(dataset_train)}")
print(f"Validation samples: {len(dataset_valid)}")

def denormalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    mean = torch.tensor(mean).view(1, 3, 1, 1).to(tensor.device)
    std = torch.tensor(std).view(1, 3, 1, 1).to(tensor.device)
    return tensor * std + mean

def train(model, dataloader):
    model.train()
    running_loss = 0.0
    running_psnr = 0.0
    for bi, data in tqdm(enumerate(dataloader), total=len(dataloader)):
        image_data = data[0].to(device)
        label = data[1].to(device)
        
        # Zero grad the optimizer.
        optimizer.zero_grad()
        outputs = model(image_data)

        outputs_denorm = denormalize(outputs)

        loss = criterion(outputs_denorm, label)
        
        # Backpropagation.
        loss.backward()
        # Update the parameters.
        optimizer.step()
        
        # Add loss of each item (total items in a batch = batch size).
        running_loss += loss.item()
        # Calculate batch psnr (once every `batch_size` iterations).
        batch_psnr =  psnr(label, outputs_denorm)
        running_psnr += batch_psnr
    
    final_loss = running_loss/len(dataloader.dataset)
    final_psnr = running_psnr/len(dataloader)
    return final_loss, final_psnr

def validate(model, dataloader, epoch):
    model.eval()
    running_loss = 0.0
    running_psnr = 0.0
    with torch.no_grad():
        for bi, data in tqdm(enumerate(dataloader), total=len(dataloader)):
            image_data = data[0].to(device)
            label = data[1].to(device)
            
            outputs = model(image_data)
            outputs_denorm = denormalize(outputs)
            loss = criterion(outputs_denorm, label)
            
            # Add loss of each item (total items in a batch = batch size) .
            running_loss += loss.item()
            
            # Calculate batch psnr (once every `batch_size` iterations).
            batch_psnr = psnr(label, outputs_denorm)
            running_psnr += batch_psnr
            
            # For saving the batch samples for the validation results
            # every 500 epochs.
            if SAVE_VALIDATION_RESULTS and (epoch % 500) == 0:
                save_validation_results(outputs_denorm, epoch, bi)
    final_loss = running_loss/len(dataloader.dataset)
    final_psnr = running_psnr/len(dataloader)
    return final_loss, final_psnr

train_loss, val_loss = [], []
train_psnr, val_psnr = [], []
start = time.time()
for epoch in range(epochs):
    print(f"Epoch {epoch + 1} of {epochs}")
    train_epoch_loss, train_epoch_psnr = train(model, train_loader)
    val_epoch_loss, val_epoch_psnr = validate(model, valid_loader, epoch+1)
    print(f"Train PSNR: {train_epoch_psnr:.3f}")
    print(f"Val PSNR: {val_epoch_psnr:.3f}")
    train_loss.append(train_epoch_loss)
    train_psnr.append(train_epoch_psnr)
    val_loss.append(val_epoch_loss)
    val_psnr.append(val_epoch_psnr)
    
    # Save model with all information every 100 epochs. Can be used 
    # resuming training.
    if (epoch+1) % 100 == 0:
        save_model(epoch, model, optimizer, criterion)
    
    # Save the model state dictionary only every epoch. Small size, 
    # can be used for inference.
    save_model_state(model)
    
    # Save the PSNR and loss plots every epoch.
    save_plot(train_loss, val_loss, train_psnr, val_psnr)
end = time.time()
print(f"Finished training in: {((end-start)/60):.3f} minutes") 

# Testing

Test loading and upscaling a single image.

In [132]:
train_images = next(iter(train_loader))
valid_images = next(iter(valid_loader))

def np_denormalize(img):
    img = img.numpy().transpose((1,2,0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1)
    return img

In [ ]:
print(len(valid_images))
print(valid_images[0].shape)

lr_image = np_denormalize(valid_images[0][0])
hr_image = np_denormalize(valid_images[1][0])

with torch.no_grad():
    sr_image = model(valid_images[0].to(device)).cpu()[0]
sr_image = np_denormalize(sr_image)

plt.figure(figsize=(20,10))
plt.subplot(1,3,1)
plt.title("Low Resolution (Bicubic Scaling)")
plt.imshow(lr_image)
plt.subplot(1,3,2)
plt.title("Super Resolution (CNN Scaling)")
plt.imshow(sr_image)
plt.subplot(1,3,3)
plt.title("High Resolution (Label)")
plt.imshow(hr_image)
plt.show()



Actual testing is currently broken because of the rescaling performed in the preprocessing inside the model.

In [46]:
import torch
import glob as glob
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from PIL import Image
from torch.utils.data import DataLoader, Dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def validate(model, dataloader, device):
    model.eval()
    running_loss = 0.0
    running_psnr = 0.0
    with torch.no_grad():
        for bi, data in tqdm(enumerate(dataloader), total=len(dataloader)):
            image_data = data[0].to(device)
            label = data[1].to(device)
            outputs = model(image_data)
            # Calculate batch psnr (once every `batch_size` iterations).
            batch_psnr = psnr(label, outputs)
            running_psnr += batch_psnr

    final_loss = running_loss/len(dataloader.dataset)
    final_psnr = running_psnr/len(dataloader)
    return final_loss, final_psnr

# The SRCNN dataset module.
class SRCNNDataset(Dataset):
    def __init__(self, image_paths):
        self.all_image_paths = glob.glob(f"{image_paths}/*")

    def __len__(self):
        return (len(self.all_image_paths))

    def __getitem__(self, index):
        # The high resolution ground truth label.
        label = Image.open(self.all_image_paths[index]).convert('RGB')
        w, h = label.size[:]
        # Convert to 2x bicubic.
        low_res_img = label.resize((int(w*0.5), int(h*0.5)), Image.BICUBIC)
        # The low resolution input image.
        image = low_res_img.resize((w, h), Image.BICUBIC)

        # Uncomment the below code to visualize the image for sanity check.
        #plt.figure(figsize=(12, 9))
        #plt.subplot(1, 2, 1)
        #plt.imshow(image)
        #plt.axis('off')
        #plt.subplot(1, 2, 2)
        #plt.imshow(label)
        #plt.axis('off')
        #plt.show()

        image = np.array(image, dtype=np.float32)
        label = np.array(label, dtype=np.float32)

        image /= 255.
        label /= 255.

        image = image.transpose([2, 0, 1])
        label = label.transpose([2, 0, 1])

        return (
            torch.tensor(image, dtype=torch.float),
            torch.tensor(label, dtype=torch.float)
        )

# Prepare the datasets.
def get_datasets(
    image_paths
):
    dataset_test = SRCNNDataset(image_paths)
    return dataset_test

# Prepare the data loaders
def get_dataloaders(dataset_test):
    test_loader = DataLoader(
        dataset_test, 
        batch_size=1,
        shuffle=False
    )
    return test_loader

In [ ]:
# Load the model.
model = ResNetSRCNN().to(device)
model.load_state_dict(torch.load('outputs/model.pth'))
data_paths = [
    ['input/Set5/original', 'Set5'],
    ['input/Set14/original', 'Set14']
]
for data_path in data_paths:
    dataset_test = get_datasets(data_path[0])
    test_loader = get_dataloaders(dataset_test)
    _, test_psnr = validate(model, test_loader, device)
    print(f"Test PSNR on {data_path[1]}: {test_psnr:.3f}")